In [1]:
# Path to the metadata file
metadata_path = r"\\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\RPAMetadata.xlsx"

# Temporary LOCAL location
output_path = r"D:\RTBtemp"

# Final destination (ecology sharedrive)
final_path = r"\\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\PPKCorrectedFlightData"

# Define the Redcatchtoolbox CLI executable path (quoted and raw string to handle spaces)
REDTOOLBOX_CLI = r'"C:\Program Files\REDToolBox\resources\assets\REDtoolboxCLI\REDtoolboxCLI.exe"'
ATX_PATH = r"C:\Program Files\REDToolBox\resources\assets\igs20.atx"
TEMP_DIR = r"C:\Users\eck025\AppData\Roaming\REDtoolbox\temp"
TEMP_MAP = r"C:\Users\eck025\AppData\Roaming\REDtoolbox\temp_mapping"

# updated 12/2025 to accept multiple obs and nav files in a 

In [2]:
# import necessary libraries - note I have uninstalled matplotlib_inline in the venv to see if this allows it to save pdfs
# there is a persistent bug exporting pdfs. I gave up as the logs showed the code is doing what i'd asked.
import glob
import json
import subprocess
from PIL import Image
from PIL.ExifTags import TAGS
from datetime import datetime, timedelta
import os
import pandas as pd
import shutil
import matplotlib
matplotlib.use('Agg')
from pathlib import Path

def run_command(args):
    """Run a CLI command with proper formatting."""
    quoted = [f'"{arg}"' if ' ' in arg or '\\' in arg else arg for arg in args]
    command = f"{REDTOOLBOX_CLI} " + " ".join(quoted)
    print(f"Running:\n{command}\n")
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print("⚠️ STDERR:\n", result.stderr)

def dms_to_decimal(dms_str):
    """Convert DMS string to decimal degrees. Handles negative degrees."""
    parts = dms_str.strip().split()

    if len(parts) != 3:
        raise ValueError(f"Invalid DMS format: {dms_str}")

    deg = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2])

    sign = -1 if deg < 0 else 1
    deg = abs(deg)

    decimal = deg + (minutes / 60) + (seconds / 3600)
    return sign * decimal

def process_flight(image_dir,output_dir, base_station_dir,base_position):
    """Process flight using RedToolbox CLI. Auto-detects required files and generates appropriate output directory."""
    # Ensure normalized Windows paths
    image_dir = str(Path(image_dir.strip()))
    base_station_dir = str(Path(base_station_dir.strip()))
    
    # Check for required drone files
    mrk_files = glob.glob(os.path.join(image_dir, "*.MRK")) + glob.glob(os.path.join(image_dir, "*.mrk"))
    mrk_file = mrk_files[0]
    
    
    rover_obs_files = glob.glob(os.path.join(image_dir, "*.obs")) + glob.glob(os.path.join(image_dir, "*.OBS"))
    rover_obs = rover_obs_files[0]
    
    # Construct output folder path using image_dir basename
    image_basename = os.path.basename(image_dir)
    output_dir = os.path.normpath(
        os.path.join(
            output_dir,
            image_basename
        )
    )
    os.makedirs(output_dir, exist_ok=True) # Ensure output directory exists

    #####################################
    # Step 1: Inspect Images
    #####################################
    run_command([
        "inspect-image",
        "-i", image_dir,
        "-ct",
        "-result", os.path.join(TEMP_DIR, "1_inspect_image.json")
    ])
    
    #####################################
    # Step 2: Inspect Trigger (MRK)
    #####################################
    run_command([
        "inspect-log",
        "-l", mrk_file,
        "-o", TEMP_DIR,
        "-result", os.path.join(TEMP_DIR, "2_inspect_log.json")
    ])
    
    #####################################
    # Step 3: Inspect Rover OBS
    #####################################
    run_command([
        "inspect-rinex",
        "-d", "dji_multispectral",
        "-r", rover_obs,
        "-o", TEMP_MAP,
        "-result", os.path.join(TEMP_DIR, "3_rover_obs.json")
    ])
    
    
    ######################################################################################
    # Extract timestamp from first image in the directory (assume jpg or png)
    ######################################################################################
    image_files = glob.glob(os.path.join(image_dir, "*.JPG")) + glob.glob(os.path.join(image_dir, "*.jpg")) + glob.glob(os.path.join(image_dir, "*.png"))
    if not image_files:
       raise FileNotFoundError("No images found in the image directory")
    timestamp = get_utc_capture_time(image_files[0])
    
    # parse the JSON to extract out the image path
    inspect_image_result = os.path.join(TEMP_DIR, "1_inspect_image.json")
    
    ############################################
    # 4. Check if obs need merging
    ############################################
    # Collect all OBS files
    obs_files = []
    obs_files += glob.glob(os.path.join(base_station_dir, "*.25O"))
    obs_files += glob.glob(os.path.join(base_station_dir, "*.obs"))
    print(obs_files)
    
    # Build the argument list for multiple `-in file`
    merge_args = ["merge-obs"]
    for nf in obs_files:
        merge_args += ["-ir", nf]
    
    # tell it where the results json (i.e., results details) go
    merged_obs_result = os.path.join(TEMP_DIR, "4_obs_merge.json")
    
    # Add output folder + result json
    merge_args += [
        "-o", output_dir,
        "-result", merged_obs_result
    ]
    
    # Execute merge
    run_command(merge_args)
    
    # extract the merge result
    with open(merged_obs_result, 'r') as f:
        data = json.load(f)
    merged_obs_path = data["merged_obs_path"]
    
    print("obs merge completed")
    print("Merged file in: ",merged_obs_path)
    print("Result JSON:", merged_obs_result)
    
    # inspect rinex files
    run_command([
        "inspect-rinex",
        "-d", "dji_multispectral",
        "-b", merged_obs_path,
        "-o", TEMP_MAP,
        "-result", os.path.join(TEMP_DIR, "5_base_obs.json")
    ])


    ##################################
    # 5. Check if nav need merging
    ##################################
    # Collect all NAV files
    nav_files = []
    nav_files += glob.glob(os.path.join(base_station_dir, "*.25P"))
    nav_files += glob.glob(os.path.join(base_station_dir, "*.nav"))
    
    if len(nav_files) == 0:
        raise RuntimeError("No NAV files found in any base station directories")
    
    # Build the argument list for multiple `-in file`
    merge_args = ["merge-nav"]
    for nf in nav_files:
        merge_args += ["-in", nf]
    
    # tell it where the results json (i.e., results details) go
    merged_nav_result = os.path.join(TEMP_DIR, "6_nav_merge.json")
    
    # Add output folder + result json
    merge_args += [
        "-o", output_dir,
        "-result", merged_nav_result
    ]
    
    # Execute merge
    run_command(merge_args)
    
    # extract the merge result
    with open(merged_nav_result, 'r') as f:
        data = json.load(f)
    merged_nav_path = data["merged_nav_path"]
    
    print("NAV merge completed")
    print("Merged file in: ",merged_nav_path)
    print("Result JSON:", merged_nav_result)

    #############################################
    # Step 7: Final Mapping
    #############################################
    run_command([
        "mapping",
        "-d", "dji_multispectral",
        "-ct", "ppk",
        "-of", "textfile",
        "-of", "pdf",
        "-of", "exif",
        "-of", "pix4d",
        #"-of", "agisoft",
        "-i", image_dir,
        "-l", mrk_file,
        "-r", rover_obs,
        "-b", merged_obs_path,
        "-nav", merged_nav_path,
        "-bpt", *base_position,
        "-mapfile",
        "-atx", ATX_PATH,
        "-o", output_dir,
        "-result", os.path.join(TEMP_DIR, "7_mapping_result.json")
    ])
    
    print(f"\n Processing complete. Output saved to:\n{output_dir}")
    
    # move the output directory to the final destination (sharedrive)
    final_output_dir = os.path.join(final_path, image_basename)
    
    print(f"\n Copying output to final destination:\n{final_output_dir}")
    shutil.copytree(output_dir, final_output_dir, dirs_exist_ok=True)
    
    # Delete the temporary local output directory
    print(f"\n🧹 Cleaning up local output directory:\n{output_dir}")
    shutil.rmtree(output_dir)
    
    print("\n Output successfully moved and temp file deleted")


In [4]:
# Read in flights= metadata
FlightsDF = pd.read_excel(metadata_path, sheet_name=0)
AusPosDF = pd.read_excel(metadata_path, sheet_name=1)

# Strip file extension from Base_log_path and merge
FlightsDF["Base_filename"] = FlightsDF["Base_log_path"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
FlightsDF["Base_filename"] = FlightsDF["Base_filename"].str.replace(r"_RINEX_\d_\d{2}", "", regex=True)

# Merge on the file key
merged_df = FlightsDF.merge(AusPosDF, left_on="Base_filename", right_on="File")

# Filter out dodgy flights
merged_df = merged_df[
    (merged_df["Superseded"] != True) &
    (merged_df["Base_corrected_Auspos"] == True) &
    (merged_df["Latitude_DMS"].notna()) &
    (merged_df["Longitude_DMS"].notna())
]

# Convert DMS coordinates to decimal degrees
merged_df["Latitude_dd"] = merged_df["Latitude_DMS"].apply(dms_to_decimal)
merged_df["Longitude_dd"] = merged_df["Longitude_DMS"].apply(dms_to_decimal)

In [ ]:
exif_paths = []

for i in range(len(merged_df)):
    row = merged_df.iloc[i]
    image_dir = row["Raw_data_dir"].split(';')
    base_station_dir = row["Base_log_path"]
    lat = str(row["Latitude_dd"])
    lon = str(row["Longitude_dd"])
    ellh = str(row["Ellipsoidal_Height_m"])

    print(f"▶️ Processing flight {i + 1} of {len(merged_df)}...")

    processed_output_dirs = []

    for j in range(len(image_dir)):
        image_subdir = os.path.basename(image_dir[j])
        final_output_dir = os.path.join(final_path, image_subdir)

        if os.path.exists(final_output_dir):
            print(f"⏭️ Skipping {image_dir[j]} — already processed.")
            processed_output_dirs.append(final_output_dir)
            continue

        try:
            process_flight(
                image_dir=image_dir[j],
                output_dir=output_path,
                base_station_dir=base_station_dir,
                base_position=[lat, lon, ellh]
            )
            processed_output_dirs.append(final_output_dir)

        except Exception as e:
            print(f"❌ Error processing flight {image_dir[j]}: {e}")
            # If any part of a multi-dir flight fails, skip the whole row
            processed_output_dirs = []
            break

    # Add joined list of output directories or empty string
    if processed_output_dirs:
        exif_paths.append(";".join(processed_output_dirs))
    else:
        exif_paths.append("")

merged_df["ExifCorrectedImagePath"] = exif_paths

merged_df.to_csv(os.path.join(final_path, "ProcessedRPAMetadata.csv"), index=False)


▶️ Processing flight 1 of 50...
⏭️ Skipping \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\RawFlightData\DJI_202505091014_002_CR-F05-60m-a — already processed.
⏭️ Skipping \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\RawFlightData\DJI_202505091041_003_CR-F05-60m-b — already processed.
▶️ Processing flight 2 of 50...
⏭️ Skipping \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\RawFlightData\DJI_202505091052_004_CR-F04-40m-a — already processed.
⏭️ Skipping \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\RawFlightData\DJI_202505091122_006_CR-F04-40m-b — already processed.
▶️ Processing flight 3 of 50...
⏭️ Skipping \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\RawFlightData\DJI_202505091148_001_CR-F02-60m-a — already processed.
⏭️ Skippi